In [1]:
# parameters 
data_version='dm_gm_public_maize_232'
port='http://api-dev:8080/genomagic-api/v1/'
hap_markers_job='sync-job/HAPLOTYPE_MARKERS_COLOR_VCF.vcf'
hap_similarity_job='sync-job/HAPLOTYPES_SIMILARITY_COLOR_VCF.vcf'
output_file='temp'
log_file='log.log'
minBinSize=100000



In [2]:
%%timeit -r1 -n1
from urllib2 import Request, urlopen, URLError
import MinimalMarkerSetUtils

# code
samples_url='{}samples?dataVersion={}&analysisMethods=applied_reference_genome,whole_genome_sequencing&responseType=tsv'.format(port,data_version)
samples_data=urlopen(Request(samples_url)).read().rstrip().split('\n')
samples_num=len(samples_data)
for i in range(samples_num):
    samples_data[i]=samples_data[i].split('\t')[0]
similarity_url='{}{}?dataVersion={}&samples={}&minBinSize={}'.format(port,hap_similarity_job,data_version,','.join(samples_data),minBinSize)   
markers_url='{}{}?dataVersion={}&locations=1,2,3,4,5,6,7,8,9,10&samplesQuery={}&outputSamples={}'.format(port,hap_markers_job,data_version,'+OR+'.join(samples_data),','.join(samples_data))
similarity_request = urlopen(Request(similarity_url))
markers_request = urlopen(Request(markers_url))
[similarity_data,similarity_pos,sample_names]=MinimalMarkerSetUtils.readHaplotypeSimilarityVcfFile(similarity_request)
output=open(output_file,'w')
logfile=open(log_file,'w')
logfile.write('chr\tstart\tend\t\n')
[total_unmatched,total_haps]=MinimalMarkerSetUtils.iterateOverSimilarities(similarity_data,similarity_pos,sample_names,markers_request,output,logfile)
output.close()
logfile.close()
print "{} haplotypes were not matched out of {}".format(total_unmatched,total_haps)

IndexError: list index out of range